In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly as py
from plotly.offline import init_notebook_mode, iplot 
init_notebook_mode(connected=True) 
import plotly.graph_objs as go
from math import sqrt

In [14]:
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [15]:
import pandas as pd

df = pd.read_csv('dst_data.csv')
df['Дата'] = df['Дата'].replace({'24:00':'23:59'}, regex=True)
df

,Дата,Базисное значение,Среднесуточное значение,Текущее значение
0,1957-01-01 0:00,0,4,11
1,1957-01-01 1:00,0,4,13
2,1957-01-01 2:00,0,4,12
3,1957-01-01 3:00,0,4,12
4,1957-01-01 4:00,0,4,9
...,...,...,...,...
529595,2014-12-31 20:00,0,-17,-27
529596,2014-12-31 21:00,0,-17,-27
529597,2014-12-31 22:00,0,-17,-23
529598,2014-12-31 23:00,0,-17,-23


In [16]:
df['Дата'] = pd.to_datetime(df['Дата'])

In [17]:
df = df.iloc[429150:]

In [18]:
timeSeries = df.loc[:, ['Дата', 'Текущее значение']]
timeSeries.index = timeSeries['Дата']
ts = timeSeries.drop(['Дата'],axis=1)

In [19]:
ts = ts.iloc[-10000:]

In [53]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [54]:

window = 29
series = series_to_supervised(train_gp.drop('date', axis=1), window=window, lag=lag)
series.head()

NameError: name 'lag_size' is not defined

In [ ]:

last_item = 'item(t-%d)' % window
last_store = 'store(t-%d)' % window
series = series[(series['store(t)'] == series[last_store])]
series = series[(series['item(t)'] == series[last_item])]


columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['item', 'store']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['item', 'store']]
series.drop(columns_to_drop, axis=1, inplace=True)
series.drop(['item(t)', 'store(t)'], axis=1, inplace=True)

labels_col = 'sales(t+%d)' % lag_size
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

* First we will use a Multilayer Perceptron model or MLP model, here our model will have input features equal to the window size.
* The thing with MLP models is that the model don't take the input as sequenced data, so for the model, it is just receiving inputs and don't treat them as sequenced data, that may be a problem since the model won't see the data with the sequence patter that it has.
* Input shape **[samples, timesteps]**.

epochs = 40
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

model_mlp = Sequential()
model_mlp.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))
model_mlp.add(Dense(1))
model_mlp.compile(loss='mse', optimizer=adam)
model_mlp.summary()

mlp_history = model_mlp.fit(X_train.values, Y_train, validation_data=(X_valid.values, Y_valid), epochs=epochs, verbose=2)

* For the CNN model we will use one convolutional hidden layer followed by a max pooling layer. The filter maps are then flattened before being interpreted by a Dense layer and outputting a prediction.
* The convolutional layer should be able to identify patterns between the timesteps.
* Input shape **[samples, timesteps, features]**.

* Reshape from [samples, timesteps] into [samples, timesteps, features].
* This same reshaped data will be used on the CNN and the LSTM model.